In [62]:
import numpy as np
import pandas as pd
import seaborn as sns
import random

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

pd.set_option("display.max_rows", None)

In [2]:
df = pd.read_csv('raw_dataset_remain_missing_value.csv', index_col = 0)
df.head()

,Brand,CPU Name,CPU Mark,GPU Name,GPU Mark,Monitor,Width,Height,RAM,Storage Amount,Storage Type,OS,Weight,Price
0,MSI,Intel Core i7 13620H,26454.0,GeForce RTX 2050,7320.0,15.6,1920.0,1080.0,16384.0,2097152.0,SSD,Windows 11,3.03,1529.00
3,Dell,Intel Core i7 1360P,19333.0,Intel Iris Pro Graphics 6200,1449.0,16.0,1920.0,1200.0,24576.0,1048576.0,SSD,Windows 10,NaN,1079.00
4,Dell,Intel Core i7 10700 2.90GHz,16550.0,Intel Arc A730M,8274.0,15.6,1920.0,1200.0,32768.0,1048576.0,SSD,Windows 10,1.92,1907.00
5,Asus,AMD Ryzen 9 7945HX,55651.0,GeForce RTX 4070,26876.0,17.3,2560.0,1400.0,16384.0,1048576.0,SSD,Windows 11,2.68,1959.99
6,Lenovo,AMD Ryzen 9 6900HX,24887.0,GeForce RTX 3070,22459.0,16.0,2560.0,1600.0,16384.0,1048576.0,SSD,Windows 11,NaN,1499.99


In [3]:
# missing_brand_indices = merged_df.index[merged_df['Brand'].isna()].tolist()
# # print(len(missing_brand_indices))

# for i in missing_brand_indices:
#     merged_df = merged_df.drop(i)

In [4]:
# merged_df.reset_index(inplace=True)
# merged_df

In [5]:
pd.Series(df.isnull().sum())[pd.Series(df.isnull().sum()) > 0].sort_values(ascending=False)

Weight            604
Storage Amount     85
Storage Type       85
Width              34
Height             34
RAM                16
GPU Name            8
GPU Mark            8
CPU Name            1
CPU Mark            1
Monitor             1
dtype: int64

# Remove blank CPU and GPU

In [7]:
data = df[(pd.notna(df['CPU Name'])) & (pd.notna(df['GPU Name']))]
data.shape

(2902, 14)

In [8]:
pd.Series(data.isnull().sum())[pd.Series(data.isnull().sum()) > 0].sort_values(ascending=False)

Weight            604
Storage Amount     85
Storage Type       85
Width              34
Height             34
RAM                16
Monitor             1
dtype: int64

# Define the specific NaN filling function

In [6]:
def fill_specific_na(x):
    global i
    if not (pd.isna(x)):
        return x
    i+=1
    return fillna_arr[i]

# Storage Type

In [9]:
data['Storage Type'].value_counts()

SSD       2796
Hybrid      21
Name: Storage Type, dtype: int64

In [13]:
data[pd.isna(data['Storage Type'])].head()

,Brand,CPU Name,CPU Mark,GPU Name,GPU Mark,Monitor,Width,Height,RAM,Storage Amount,Storage Type,OS,Weight,Price
8,Core Innovations,Intel Celeron N3350 1.10GHz,1108.0,Intel UHD Graphics 600,324.0,14.1,1366.0,768.0,3072.0,NaN,NaN,Windows 10,1.59,99.00
16,Samsung,Intel Core i3 10110U 2.10GHz,3919.0,Intel UHD Graphics,1476.0,13.3,1920.0,1080.0,16384.0,NaN,NaN,ChromeOS,1.68,689.99
103,Asus,Intel Celeron N3350 1.10GHz,1108.0,Intel UHD Graphics 600,324.0,11.6,1366.0,768.0,4096.0,NaN,NaN,Windows 10,1.27,342.00
123,LG,Intel Core i7 1360P,19333.0,Intel Iris Pro Graphics 6200,1449.0,15.6,1920.0,1200.0,16384.0,NaN,NaN,Windows 11,1.14,1171.99
142,Asus,AMD Ryzen 5 1400,7773.0,Radeon Vega 8,1585.0,15.6,1920.0,1080.0,8192.0,NaN,NaN,Windows 10,1.68,427.00


In [10]:
fillna_arr = np.random.choice(np.arange(len(data['Storage Type'].value_counts().index)), 
                              size=len(data[data['Storage Type'].isna()]['Storage Type']), 
                              p=data['Storage Type'].value_counts().values/sum(data['Storage Type'].value_counts().values))

In [11]:
i=-1
data['Storage Type'] = data['Storage Type'].apply(fill_specific_na).replace(to_replace=np.arange(len(data['Storage Type'].value_counts().index)), 
                                                                                        value=data['Storage Type'].value_counts().index)
data['Storage Type'].value_counts()

C:\Users\DELL\AppData\Local\Temp/ipykernel_11588/1450398707.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Storage Type'] = data['Storage Type'].apply(fill_specific_na).replace(to_replace=np.arange(len(data['Storage Type'].value_counts().index)),


SSD       2880
Hybrid      22
Name: Storage Type, dtype: int64

In [17]:
impute_stt_indices = data[pd.isna(data['Storage Type'])].index
len(impute_stt_indices)

85

In [18]:
for idx in impute_stt_indices:
    data['Storage Type'][idx] = np.random.choice(['SSD','Hybrid'])

C:\Users\DELL\AppData\Local\Temp/ipykernel_19436/2830344528.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Storage Type'][idx] = np.random.choice(['SSD','Hybrid'])


,Brand,CPU Name,CPU Mark,GPU Name,GPU Mark,Monitor,Width,Height,RAM,Storage Amount,Storage Type,OS,Weight,Price
0,MSI,Intel Core i7 13620H,26454.0,GeForce RTX 2050,7320.0,15.6,1920.0,1080.0,16384.0,2097152.0,SSD,Windows 11,3.03,1529.00
3,Dell,Intel Core i7 1360P,19333.0,Intel Iris Pro Graphics 6200,1449.0,16.0,1920.0,1200.0,24576.0,1048576.0,SSD,Windows 10,NaN,1079.00
4,Dell,Intel Core i7 10700 2.90GHz,16550.0,Intel Arc A730M,8274.0,15.6,1920.0,1200.0,32768.0,1048576.0,SSD,Windows 10,1.92,1907.00
5,Asus,AMD Ryzen 9 7945HX,55651.0,GeForce RTX 4070,26876.0,17.3,2560.0,1400.0,16384.0,1048576.0,SSD,Windows 11,2.68,1959.99
6,Lenovo,AMD Ryzen 9 6900HX,24887.0,GeForce RTX 3070,22459.0,16.0,2560.0,1600.0,16384.0,1048576.0,SSD,Windows 11,NaN,1499.99


In [21]:
data['Storage Type'].value_counts()

SSD       2837
Hybrid      65
Name: Storage Type, dtype: int64

# Storage Amount

In [ ]:
# Fill missing values with median
data['Storage Amount'] = data['Storage Amount'].fillna(data.groupby('Storage Type')['Storage Amount'].transform('median')).ffill()

In [14]:
data['Storage Amount'].value_counts()

5.242880e+05    820
1.048576e+06    818
2.621440e+05    300
2.097152e+06    296
1.024000e+06    236
4.194304e+06    107
1.310720e+05     95
2.048000e+06     61
6.553600e+04     53
3.276800e+04     35
5.120000e+05     25
4.096000e+06     15
8.388608e+06      8
3.145728e+06      7
3.276800e+05      4
8.192000e+03      3
1.638400e+04      2
1.024000e+03      2
4.915200e+05      2
1.000000e+00      1
6.656000e+04      1
2.621440e+06      1
2.457600e+05      1
1.048576e+09      1
1.310720e+06      1
2.560000e+05      1
6.400000e+01      1
1.572864e+06      1
1.286144e+06      1
1.677722e+07      1
2.560000e+02      1
1.024000e+05      1
Name: Storage Amount, dtype: int64

# Monitor, Weight, Height

In [17]:
data[data['Monitor'].isna()] = 14.0

C:\Users\DELL\AppData\Local\Temp/ipykernel_11588/1232072579.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[data['Monitor'].isna()] = 14.0


In [18]:
data['Monitor'].isna().sum()

0

In [21]:
# Fill missing values with median
data['Width'] = data['Width'].fillna(data.groupby('Monitor')['Width'].transform('median')).ffill()
data['Height'] = data['Height'].fillna(data.groupby(['Monitor', 'Width'])['Width'].transform('median')).ffill()

C:\Users\DELL\AppData\Local\Temp/ipykernel_11588/539248799.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Width'] = data['Width'].fillna(data.groupby('Monitor')['Width'].transform('median')).ffill()
C:\Users\DELL\AppData\Local\Temp/ipykernel_11588/539248799.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Height'] = data['Height'].fillna(data.groupby(['Monitor', 'Width'])['Width'].transform('median')).ffill()


In [23]:
data['Monitor'].value_counts()

15.6    1354
14.0     493
16.0     377
17.3     331
17.0      94
13.3      87
11.6      39
16.1      30
13.5      15
15.0      13
13.4      12
14.5      10
13.0       8
14.1       6
18.0       6
10.1       5
12.0       3
12.3       3
13.1       3
12.5       3
3.0        2
12.2       2
8.0        1
10.5       1
13.7       1
14.4       1
13.9       1
12.4       1
Name: Monitor, dtype: int64

# RAM

In [19]:
# Fill missing values with median
data['RAM'] = data['RAM'].fillna(data.groupby(['CPU Name','GPU Name'])['RAM'].transform('median')).ffill()

C:\Users\DELL\AppData\Local\Temp/ipykernel_11588/1148197557.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['RAM'] = data['RAM'].fillna(data.groupby(['CPU Name','GPU Name'])['RAM'].transform('median')).ffill()


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 4105
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Brand           2902 non-null   object 
 1   CPU Name        2902 non-null   object 
 2   CPU Mark        2902 non-null   float64
 3   GPU Name        2902 non-null   object 
 4   GPU Mark        2902 non-null   float64
 5   Monitor         2902 non-null   float64
 6   Width           2868 non-null   float64
 7   Height          2868 non-null   float64
 8   RAM             2902 non-null   float64
 9   Storage Amount  2902 non-null   float64
 10  Storage Type    2902 non-null   object 
 11  OS              2902 non-null   object 
 12  Weight          2299 non-null   object 
 13  Price           2902 non-null   float64
dtypes: float64(8), object(6)
memory usage: 340.1+ KB


# Weight

In [28]:
data['New Weight'] = data['Weight']

C:\Users\DELL\AppData\Local\Temp/ipykernel_11588/4237878508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['New Weight'] = data['Weight']


In [30]:
data.head()

,Brand,CPU Name,CPU Mark,GPU Name,GPU Mark,Monitor,Width,Height,RAM,Storage Amount,Storage Type,OS,Weight,Price,New Weight
0,MSI,Intel Core i7 13620H,26454.0,GeForce RTX 2050,7320.0,15.6,1920.0,1080.0,16384.0,2097152.0,SSD,Windows 11,3.03,1529.00,3.03
3,Dell,Intel Core i7 1360P,19333.0,Intel Iris Pro Graphics 6200,1449.0,16.0,1920.0,1200.0,24576.0,1048576.0,SSD,Windows 10,NaN,1079.00,NaN
4,Dell,Intel Core i7 10700 2.90GHz,16550.0,Intel Arc A730M,8274.0,15.6,1920.0,1200.0,32768.0,1048576.0,SSD,Windows 10,1.92,1907.00,1.92
5,Asus,AMD Ryzen 9 7945HX,55651.0,GeForce RTX 4070,26876.0,17.3,2560.0,1400.0,16384.0,1048576.0,SSD,Windows 11,2.68,1959.99,2.68
6,Lenovo,AMD Ryzen 9 6900HX,24887.0,GeForce RTX 3070,22459.0,16.0,2560.0,1600.0,16384.0,1048576.0,SSD,Windows 11,NaN,1499.99,NaN


In [71]:
print(data['New Weight'][7])

2.55


In [65]:
def convert_ounces(weight):
    try:
        weight = weight.lower()
        if 'ounces' in weight:
            res = float(weight.replace('ounces','').strip())
            return '%.2f' % (res / 35)
        else:
            return weight
    except:
        return None

odd_idx = data[(pd.notna(data['New Weight'])) & (data['New Weight'].str.contains('Ounces'))].index
odd_idx

Int64Index([], dtype='int64')

In [58]:
for idx in odd_idx:
    data['New Weight'][idx] = convert_ounces(data['New Weight'][idx])

C:\Users\DELL\AppData\Local\Temp/ipykernel_11588/3479617452.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['New Weight'][idx] = convert_ounces(data['New Weight'][idx])


In [78]:
data['New Weight'] = data['New Weight'].astype(float)

C:\Users\DELL\AppData\Local\Temp/ipykernel_11588/3913581932.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['New Weight'] = data['New Weight'].astype(float)


In [79]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 4105
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Brand           2902 non-null   object 
 1   CPU Name        2902 non-null   object 
 2   CPU Mark        2902 non-null   float64
 3   GPU Name        2902 non-null   object 
 4   GPU Mark        2902 non-null   float64
 5   Monitor         2902 non-null   float64
 6   Width           2902 non-null   float64
 7   Height          2902 non-null   float64
 8   RAM             2902 non-null   float64
 9   Storage Amount  2902 non-null   float64
 10  Storage Type    2902 non-null   object 
 11  OS              2902 non-null   object 
 12  Weight          2299 non-null   object 
 13  Price           2902 non-null   float64
 14  New Weight      2299 non-null   float64
dtypes: float64(9), object(6)
memory usage: 427.3+ KB


In [80]:
data['New Weight'] = data['New Weight'].interpolate(method='linear').round(2)
data['New Weight'].isna().sum()

C:\Users\DELL\AppData\Local\Temp/ipykernel_11588/3491790448.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['New Weight'] = data['New Weight'].interpolate(method='linear').round(2)


0

In [90]:
column_to_move = data['New Weight']

data.insert(12, 'Weight', column_to_move)

In [92]:
del data['New Weight']

In [93]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 4105
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Brand           2902 non-null   object 
 1   CPU Name        2902 non-null   object 
 2   CPU Mark        2902 non-null   float64
 3   GPU Name        2902 non-null   object 
 4   GPU Mark        2902 non-null   float64
 5   Monitor         2902 non-null   float64
 6   Width           2902 non-null   float64
 7   Height          2902 non-null   float64
 8   RAM             2902 non-null   float64
 9   Storage Amount  2902 non-null   float64
 10  Storage Type    2902 non-null   object 
 11  OS              2902 non-null   object 
 12  Weight          2902 non-null   float64
 13  Price           2902 non-null   float64
dtypes: float64(9), object(5)
memory usage: 404.6+ KB


# Export CSV

In [94]:
data.to_csv("missing_value_free.csv")